# Load data from the Rosstat files and output as long-format CSV

In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd

In [2]:
def rename(regions):
    regions[regions=='Еврейская автономная область'] = 'Еврейская АО'
    regions[regions=='Республика Северная Осетия-Алания'] = 'Северная Осетия'
    regions[regions=='Республика Саха (Якутия)'] = 'Якутия'
    regions[regions=='Кабардино-Балкарская Республика'] = 'Кабардино-Балкария'
    regions[regions=='Карачаево-Черкесская Республика'] = 'Карачаево-Черкесия'
    
    regions[regions=='г.Москва'] = 'Москва'
    regions[regions=='г. Москва'] = 'Москва'
    regions[regions=='г.Санкт-Петербург'] = 'Санкт-Петербург'
    regions[regions=='г. Санкт-Петербург'] = 'Санкт-Петербург'
    regions[regions=='г.Севастополь'] = 'Севастополь'
    regions[regions=='г. Севастополь'] = 'Севастополь'
    
    regions[regions=='Ненецкий авт. округ'] = 'Ненецкий АО'
    regions[regions=='Архангельская область без автономии'] = 'Архангельская область без АО'
    regions[regions=='Ханты-Мансийский авт. округ - Югра'] = 'Ханты-Мансийский АО'
    regions[regions=='Ямало-Ненецкий авт. округ'] = 'Ямало-Hенецкий АО'
    regions[regions=='Тюменская область без авт.округов'] = 'Тюменская область без АО'
    regions[regions=='Чукотский автономный округ'] = 'Чукотский АО'
    
    return regions

In [3]:
file = 'deaths-by-age-gender-region-year/1990-2021.xlsx'

sheet_names = ['1990-2001', '2002-2013', '2014-2021']

df_out = pd.DataFrame(columns=['Region', 'Year', 'Age', 'Gender', 'Deaths'])

for sheet_name in sheet_names:
    df = pd.read_excel(file, sheet_name=sheet_name)
    
    years = np.arange(int(sheet_name[:4]), int(sheet_name[-4:]) + 1)
    first_column = df.values[:,0].copy().astype(str)
    first_column = [fc[:4] if len(fc)>=4 else '' for fc in first_column]
    year_idx = np.where(np.isin(first_column, years.astype(str)))[0]
    assert(year_idx.size == years.size)
    
    age_bands = df.values[2, 1:]
    genders = ['m'] * (age_bands.size//2) + ['f'] * (age_bands.size//2)
    genders = np.array(genders)
    
    for year_num, idx in enumerate(year_idx):
        end_idx = np.where(df.values[idx:, 0].astype('str') == 'nan')[0]
        if len(end_idx) > 0:
            end_idx = end_idx[0]
        else:
            end_idx = df.values[idx:, 0].size
    
        regions = df[idx + 1 : idx + end_idx].values[:,0]
        regions = np.array([r.strip() for r in regions])
        regions = np.array([r if r[-2:] != '1)' else r[:-2] for r in regions])

        values = df[idx + 1 : idx + end_idx].values[:,1:]
        
        # Fill in some AO values:
        if 'Архангельская область без автономии' not in regions and 'Ненецкий авт. округ' in regions:
            ao = values[regions=='Архангельская область'] - \
                 values[regions=='Ненецкий авт. округ']
            regions = np.concatenate((regions, ['Архангельская область без автономии']))
            values = np.concatenate((values, ao))

        if 'Тюменская область без авт.округов' not in regions and 'Ямало-Ненецкий авт. округ' in regions:
            ao = values[regions=='Тюменская область'] - \
                 values[regions=='Ямало-Ненецкий авт. округ'] - \
                 values[regions=='Ханты-Мансийский авт. округ - Югра']
            regions = np.concatenate((regions, ['Тюменская область без авт.округов']))
            values = np.concatenate((values, ao))
            
        # Remove empty region-years
        keep = values.sum(axis=1) > 0
        regions = regions[keep]
        values = values[keep]

        regions_replicated = np.array(list(regions) * age_bands.size).reshape([age_bands.size,-1]).T
        age_bands_replicated = np.array(list(age_bands) * regions.size).reshape(regions.size, -1)
        genders_replicated = np.array(list(genders) * regions.size).reshape(regions.size, -1)
        years_replicated = years[year_num] * np.ones((regions.size, age_bands.size)).astype(int)
    
        df_append = pd.DataFrame(
            {
                'Region': regions_replicated.flatten(), 
                'Year': years_replicated.flatten(),
                'Age': age_bands_replicated.flatten(),
                'Gender': genders_replicated.flatten(),
                'Deaths': values.flatten()
            }
        )
    
        df_out = pd.concat((df_out, df_append))
        
df = df_out.copy()

keep = ['федеральный округ' not in r for r in df.Region.values]
df = df[keep]

df.Region = rename(df.Region.values)

df.loc[df['Age'] == '85-89 лет', 'Age'] = '85 и более'
df.loc[df['Age'] == '85-89 лет', 'Gender'] = 'bla'
df.loc[df.Age == '90-94 лет', 'Age'] = '85 и более'
df.loc[df.Age == '95-99 лет', 'Age'] = '85 и более'
df.loc[df.Age == '100 и более', 'Age'] = '85 и более'
df = df.groupby(['Region', 'Year', 'Age', 'Gender']).sum().reset_index()

df_until2021 = df.copy()
df_until2021

,Region,Year,Age,Gender,Deaths
0,Алтайский край,1990,0-4 лет,f,264
1,Алтайский край,1990,0-4 лет,m,429
2,Алтайский край,1990,10-14 лет,f,28
3,Алтайский край,1990,10-14 лет,m,76
4,Алтайский край,1990,15-19 лет,f,63
...,...,...,...,...,...
109475,Ярославская область,2021,85 и более,m,1028
109476,Ярославская область,2021,Всего,f,13595
109477,Ярославская область,2021,Всего,m,11497
109478,Ярославская область,2021,Неизвестно,f,4


In [4]:
file = 'deaths-by-age-gender-region-year/2022.xlsx'

df_out = pd.DataFrame(columns=['Region', 'Year', 'Age', 'Gender', 'Deaths'])

df = pd.read_excel(file)
    
year = 2022
    
age_bands = df.values[2, 22:]
genders = ['m'] * (age_bands.size//2) + ['f'] * (age_bands.size//2)
genders = np.array(genders)

idx = 2
end_idx = 99

regions = df[idx + 1 : idx + end_idx].values[:,0]
regions = np.array([r.strip() for r in regions])
regions = np.array([r if r[-2:] != '1)' else r[:-2] for r in regions])

values = df[idx + 1 : idx + end_idx].values[:,22:]
        
# Fill in some AO values:
if 'Архангельская область без автономии' not in regions and 'Ненецкий авт. округ' in regions:
    ao = values[regions=='Архангельская область'] - \
         values[regions=='Ненецкий авт. округ']
    regions = np.concatenate((regions, ['Архангельская область без автономии']))
    values = np.concatenate((values, ao))

if 'Тюменская область без авт.округов' not in regions and 'Ямало-Ненецкий авт. округ' in regions:
    ao = values[regions=='Тюменская область'] - \
         values[regions=='Ямало-Ненецкий авт. округ'] - \
         values[regions=='Ханты-Мансийский авт. округ - Югра']
    regions = np.concatenate((regions, ['Тюменская область без авт.округов']))
    values = np.concatenate((values, ao))

regions_replicated = np.array(list(regions) * age_bands.size).reshape([age_bands.size,-1]).T
age_bands_replicated = np.array(list(age_bands) * regions.size).reshape(regions.size, -1)
genders_replicated = np.array(list(genders) * regions.size).reshape(regions.size, -1)
years_replicated = year * np.ones((regions.size, age_bands.size)).astype(int)
    
df_2022 = pd.DataFrame(
    {
        'Region': regions_replicated.flatten(), 
        'Year': years_replicated.flatten(),
        'Age': age_bands_replicated.flatten(),
        'Gender': genders_replicated.flatten(),
        'Deaths': values.flatten()
    }
)

keep = ['федеральный округ' not in r for r in df_2022.Region.values]
df_2022 = df_2022[keep]

df_2022.Region = rename(df_2022.Region.values)

df_2022.loc[df_2022.Age == 'До 1 года', 'Age'] = '0-4 лет'
df_2022.loc[df_2022.Age == '1-4 лет', 'Age'] = '0-4 лет'
df_2022 = df_2022.groupby(['Region', 'Year', 'Age', 'Gender']).sum().reset_index()

df_2022

,Region,Year,Age,Gender,Deaths
0,Алтайский край,2022,0-4 лет,f,36
1,Алтайский край,2022,0-4 лет,m,66
2,Алтайский край,2022,10-14 лет,f,18
3,Алтайский край,2022,10-14 лет,m,19
4,Алтайский край,2022,15-19 лет,f,26
...,...,...,...,...,...
3515,Ярославская область,2022,85 и более,m,807
3516,Ярославская область,2022,Всего,f,10264
3517,Ярославская область,2022,Всего,m,9202
3518,Ярославская область,2022,Неизвестно,f,1


In [6]:
df = pd.concat((df_until2021, df_2022)).reset_index(drop=True)

df.to_csv('deaths-by-age-gender-region-year-1990-2022.csv.gz', index=False)

df

,Region,Year,Age,Gender,Deaths
0,Алтайский край,1990,0-4 лет,f,264
1,Алтайский край,1990,0-4 лет,m,429
2,Алтайский край,1990,10-14 лет,f,28
3,Алтайский край,1990,10-14 лет,m,76
4,Алтайский край,1990,15-19 лет,f,63
...,...,...,...,...,...
112995,Ярославская область,2022,85 и более,m,807
112996,Ярославская область,2022,Всего,f,10264
112997,Ярославская область,2022,Всего,m,9202
112998,Ярославская область,2022,Неизвестно,f,1
